In [5]:
import numpy as np
from __future__ import print_function

## Viterbi Algorithm

In [2]:
def viterbi(pi, A, B, O):
    path = []
    S = len(pi)
    N = len(O)
    deltas = np.zeros((S, N), dtype=float)
    chosen = np.zeros((S, N), dtype=int)

    deltas[:,0] = pi * B[:,O[0]]

    for t in range(1, N):
        prob = B[:,O[t]] * (A.T * deltas[:,t-1]).T
        deltas[:,t] = np.max(prob, axis=0)
        chosen[:,t] = np.argmax(prob, axis=0)

    chosen_max = np.argmax(deltas[:,N-1])    
    path.insert (0, chosen_max)
    for t in range(N-1,0,-1):
        chosen_max = chosen[chosen_max][t]
        path.insert(0, chosen_max)
    return path

## Forward-Backward calculations

In [3]:
'''
Calculates Alpha of the forward algorithm
'''
def forward(pi, A, B, O):
    S = len(pi)
    N = len(O)
    alpha = np.zeros([S, N])
    alpha[:,0] = pi * B[:,O[0]]

    for t in range(1, N):
        alpha[:,t] = B[:,O[t]] * A.T.dot(alpha[:,t-1])

    return alpha

'''
Calculates beta of the backward algorithm
'''
def backward(pi, A, B, O):
    S = len(pi)
    N = len(O)
    beta = np.zeros([S, N])
    beta[ :, N-1] = np.ones(S)
    
    for t in range(N-2, -1, -1):
        beta[:,t] = A.dot(B[:,O[t+1]] * beta[:,t+1])
    return beta

'''
Calculates the probability of the observation sequence
'''
def seqprob_forward(alpha):    
    prob = 0.0
    prob = np.sum(alpha, axis = 0)[alpha.shape[1]-1]
    return prob

'''
Calculates the probability of the observation sequence
'''
def seqprob_backward(beta, pi, B, O):
    prob = 0.0
    prob = np.sum(beta[:,0] * pi * B[:,O[0]])
    return prob

In [8]:
# defining the hmm model
conditional = np.array([[0.4, 0.1, 0.4, 0.1],
               [0.2, 0.3, 0.2, 0.3]])
transition = np.array([[0.8, 0.2],
              [0.4, 0.6]])
initial = np.array([0.7, 0.3])
observation_map = {0: "A", 1: "C", 2: "G", 3: "T"}
observations = np.array([0, 2, 1, 2, 3, 0])
states_symbols = ["1", "2"]

N = len(observations)
O = [observations[j] for j in observations]

alpha = forward(initial, transition, conditional, O)
beta = backward(initial, transition, conditional, O)

prob1 = seqprob_forward(alpha)
prob2 = seqprob_backward(beta, initial, conditional, O)
print('Total log probability of observing the sequence %s is %g, %g.' % (observations, np.log(prob1), np.log(prob2)))

viterbi_path = viterbi(initial, transition, conditional, O)

print('Viterbi best path is ')
for j in viterbi_path:
    print(states_symbols[j], end=' ')


Total log probability of observing the sequence [0 2 1 2 3 0] is -8.16334, -8.16334.
Viterbi best path is 
1 1 1 1 1 1 